1) Sua primeira tarefa consiste em escrever uma aplicação para calcular o ganho total da empresa, o qual é obtido a partir da taxa administrativa do serviço de cartão de crédito para seus clientes. Esse ganho é calculado sobre um percentual das transações de cartão de crédito realizadas por eles. O cálculo é baseado no conjunto de dados abaixo, transacao, contrato e cliente da Figura 1.

O resultado esperado é uma consulta que retorne o ganho total da empresa por cliente que é 1.198,77 para o cliente A e 1,08 para o cliente B, conforme a Figura 2.

Código SQL utilizado

SELECT cl.nome cliente_nome, 
	ROUND(SUM((t.valor_total - (t.valor_total * t.percentual_desconto / 100))* ct.percentual / 100),2) valor
		FROM cliente cl
			INNER JOIN contrato ct
			ON ct.cliente_id = cl.cliente_id
			INNER JOIN transacao t
			ON t.contrato_id = ct.contrato_id
		WHERE ct.ativo = 1
		GROUP BY cl.nome


**Setup para o desafio**

-2) A segunda tarefa consiste em calcular o total líquido da empresa. Esse total é calculado da seguinte forma total_liquido = soma(total_bruto – desconto_percentual). O cálculo é baseado no conjunto de dados da Figura 3

O resultado esperado é uma código com pyspark que retorne o total liquido da empresa que é 59973.46.

In [ ]:
%%bash

# Instal Java
apt-get update && apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install PySpark
pip install -q pyspark

In [61]:
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [62]:
# Setup Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Desafio").getOrCreate()

In [63]:
#variavel tipo lista
transacoes = [(1,3000,6.99),
              (2, 57989,1.45),
              (4,1,None),
              (5, 34, 0.0)]


In [64]:
#sch_transacoes atribuo a estrutura de colunas
sch_transacoes = ["transacao_id","total_bruto","desconto_percentual"]
#crio o df para ele utilizar como fonte dos dados a variavél transacoes e a estrutura de colunas sch_transacoes
df = spark.createDataFrame(data=transacoes, schema = sch_transacoes)  
df.printSchema()
df.show()

root
 |-- transacao_id: long (nullable = true)
 |-- total_bruto: long (nullable = true)
 |-- desconto_percentual: double (nullable = true)

+------------+-----------+-------------------+
|transacao_id|total_bruto|desconto_percentual|
+------------+-----------+-------------------+
|           1|       3000|               6.99|
|           2|      57989|               1.45|
|           4|          1|               null|
|           5|         34|                0.0|
+------------+-----------+-------------------+



In [65]:
#Função para execução de queries
def q(query, n=50):
    return spark.sql(query).show(n=n, truncate=False)

In [ ]:
#Utilizo a função abaixo para cria a view tb_transacoes
df.createTempView("tb_transacoes")

In [67]:
q("""
    SELECT ROUND(SUM(total_bruto -(total_bruto * desconto_percentual / 100)),2)
    FROM tb_transacoes
""")

+--------------------------------------------------------------------------+
|round(sum((total_bruto - ((total_bruto * desconto_percentual) / 100))), 2)|
+--------------------------------------------------------------------------+
|59972.46                                                                  |
+--------------------------------------------------------------------------+



3) O terceiro entregável consiste na transformação de dados disponíveis em arquivo Json para o formato de dataframe, algo comum no dia a dia da empresa. Após transformar esse Json em dataframe é possível perceber que a coluna "item_list" está como dicionário. Seu gestor pediu dois pontos de atenção nessa tarefa:

Expandir a coluna num mesmo dataframe;
Normalizar os itens dessa coluna de dicionário e dividí-los em dois dataframes separados, seguindo o modelo relacional.

In [78]:
import pandas as pd


In [143]:
#importar json
json = pd.read_json("/content/data.json")
json.head()

,CreateDate,EmissionDate,Discount,NFeNumber,NFeID,ItemList
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,"[{'ProductName': 'Rice', 'Value': 35.55, 'Quan..."
1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,502,2,"[{'ProductName': 'Tomate', 'Value': 12.25, 'Qu..."
2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,503,3,"[{'ProductName': 'Beer', 'Value': 9.0, 'Quanti..."


In [144]:
dfjson = pd.DataFrame(json).explode('ItemList')
dfjson = dfjson.join(pd.json_normalize(dfjson.pop('ItemList')))
print(dfjson)


               CreateDate         EmissionDate  Discount  NFeNumber  NFeID  \
0  2021-05-24T20:21:34.79  2021-05-24T00:00:00         0        501      1   
0  2021-05-24T20:21:34.79  2021-05-24T00:00:00         0        501      1   
0  2021-05-24T20:21:34.79  2021-05-24T00:00:00         0        501      1   
1  2021-05-24T20:21:34.79  2021-05-24T00:00:00         0        502      2   
1  2021-05-24T20:21:34.79  2021-05-24T00:00:00         0        502      2   
2  2021-05-24T20:21:34.79  2021-05-24T00:00:00         0        503      3   
2  2021-05-24T20:21:34.79  2021-05-24T00:00:00         0        503      3   
2  2021-05-24T20:21:34.79  2021-05-24T00:00:00         0        503      3   

  ProductName  Value  Quantity  
0        Rice  35.55         2  
0        Rice  35.55         2  
0        Rice  35.55         2  
1       Flour  11.55         5  
1       Flour  11.55         5  
2        Bean  27.15         7  
2        Bean  27.15         7  
2        Bean  27.15         7  


In [142]:
dfjson.head()

,CreateDate,EmissionDate,Discount,NFeNumber,NFeID,ProductName,Value,Quantity
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,Rice,35.55,2
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,Rice,35.55,2
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,Rice,35.55,2
1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,502,2,Flour,11.55,5
1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,502,2,Flour,11.55,5


In [149]:
notas_fiscais = dfjson.groupby(by = 'NFeNumber')
notas_fiscais.head()

,CreateDate,EmissionDate,Discount,NFeNumber,NFeID,ProductName,Value,Quantity
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,Rice,35.55,2
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,Rice,35.55,2
0,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,501,1,Rice,35.55,2
1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,502,2,Flour,11.55,5
1,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,502,2,Flour,11.55,5
2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,503,3,Bean,27.15,7
2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,503,3,Bean,27.15,7
2,2021-05-24T20:21:34.79,2021-05-24T00:00:00,0,503,3,Bean,27.15,7
